In [2]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

In [48]:
CONTEXT_SIZE = 4
EMBEDDING_DIM = 200
# We will use Shakespeare Sonnet 2
train_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

fivegram = [((train_sentence[i], train_sentence[i + 1], train_sentence[i + 2], train_sentence[i + 3]), train_sentence[i + 4])
           for i in range(len(train_sentence) - 4)]

vocb = set(train_sentence)
vocb.add('<UNK>')
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [49]:
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 300)
        self.linear2 = nn.Linear(300, 300)
        self.linear3 = nn.Linear(300, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        log_prob = F.softmax(out, dim = 1)
        return log_prob

In [50]:
ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

In [53]:
for epoch in range(100):
    print('epoch: {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0
    for data in fivegram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        
        running_loss += loss.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))

word, label = fivegram[4]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
print(out)
prob, predict_label = torch.max(out, 1)
print(prob)
print(predict_label.item())
predict_word = idx_to_word[predict_label.item()]
print('real word is {}, predict word is {}'.format(label, predict_word))

epoch: 1
**********
Loss: -0.011666
epoch: 2
**********
Loss: -0.011669
epoch: 3
**********
Loss: -0.011672
epoch: 4
**********
Loss: -0.011675
epoch: 5
**********
Loss: -0.011678
epoch: 6
**********
Loss: -0.011681
epoch: 7
**********
Loss: -0.011684
epoch: 8
**********
Loss: -0.011687
epoch: 9
**********
Loss: -0.011690
epoch: 10
**********
Loss: -0.011693
epoch: 11
**********
Loss: -0.011695
epoch: 12
**********
Loss: -0.011698
epoch: 13
**********
Loss: -0.011701
epoch: 14
**********
Loss: -0.011704
epoch: 15
**********
Loss: -0.011707
epoch: 16
**********
Loss: -0.011710
epoch: 17
**********
Loss: -0.011713
epoch: 18
**********
Loss: -0.011716
epoch: 19
**********
Loss: -0.011719
epoch: 20
**********
Loss: -0.011722
epoch: 21
**********
Loss: -0.011725
epoch: 22
**********
Loss: -0.011728
epoch: 23
**********
Loss: -0.011731
epoch: 24
**********
Loss: -0.011733
epoch: 25
**********
Loss: -0.011736
epoch: 26
**********
Loss: -0.011739
epoch: 27
**********
Loss: -0.011742
epoch: 28


In [52]:
for i in word:
  print(idx_to_word[i.item()])

besiege
thy
brow,
And
